In [2]:
import torch

# Load emotion 
emotion = torch.load("/fsx/behavioural_computing_data/face_generation_data/AA_processed/part_22/emotions/D333_C031_1006VO_001_output_output.pt")
print(emotion.keys())
for key in emotion.keys():
    print(key)
    print(emotion[key].shape)
    


dict_keys(['labels', 'valence', 'arousal', 'scores'])
labels
torch.Size([3479])
valence
torch.Size([3479])
arousal
torch.Size([3479])
scores
torch.Size([3479, 8])


In [4]:
print(emotion["valence"].max())
print(emotion["valence"].min())
print(emotion["arousal"].max())
print(emotion["arousal"].min())

tensor(0.2467)
tensor(-0.3127)
tensor(0.4399)
tensor(-0.1810)


In [9]:
import torchvision

torchvision.io.read_video_metadata

AttributeError: module 'torchvision.io' has no attribute 'read_video_metadata'

In [8]:
import torch
import time
import torchaudio
import sys
sys.path.append("..")
from scripts.util.audio.BEATs import BEATWrapper

def load_audio(file_path, target_sr=16000):
    waveform, sr = torchaudio.load(file_path)
    print("original sr", sr)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    return waveform.squeeze(0)

def process_audio(model, audio, padding_mask, fbank_mean: float = 15.41663,
        fbank_std: float = 6.55582,
        preprocess: bool = True,
            return_embedding: bool = True,):
    start_time = time.time()
    if audio.dim() == 1:
        audio = audio.unsqueeze(0)
    with torch.no_grad():
        print(audio.shape)
        output = model.beats.extract_features(
            audio,
            padding_mask,
            fbank_mean,
            fbank_std,
            preprocess,
            return_embedding,
        )[0]
    end_time = time.time()
    return output, end_time - start_time

def compare_outputs(output1, output2, tolerance=1e-6):
    diff = torch.abs(output1 - output2)
    max_diff = torch.max(diff).item()
    mean_diff = torch.mean(diff).item()
    return max_diff <= tolerance, max_diff, mean_diff

# Load and initialize the BEATWrapper
model = BEATWrapper(fine_tuned=True, feed_as_frames=False, model_path="/data/home/antoni/code/generative-models/checkpoints/BEATs_iter3_plus_AS2M_finetuned_on_AS2M_cpt2.pt")
model.eval()

# Load an audio file
audio_path = "/fsx/behavioural_computing_data/face_generation_data/AA_processed/part_22/audio/D058_C042_07318F_001.wav"
audio = load_audio(audio_path)
print(audio.shape)

# Ensure audio length is a multiple of 2 (required by the wrapper)
if audio.size(0) % 2 != 0:
    audio = audio[:-1]

audio = audio[:16000]
# Process with None padding mask
output_none, time_none = process_audio(model, audio, padding_mask=None)

# Process with zero tensor padding mask
zero_mask = torch.zeros((1, audio.size(0))).bool()
output_zero, time_zero = process_audio(model, audio, padding_mask=zero_mask)

# Compare results
is_equal, max_diff, mean_diff = compare_outputs(output_none, output_zero)

print(f"Outputs are {'equal' if is_equal else 'different'}")
print(f"Max difference: {max_diff}")
print(f"Mean difference: {mean_diff}")
print(f"Time with None mask: {time_none:.6f} seconds")
print(f"Time with zero mask: {time_zero:.6f} seconds")
print(f"Time difference: {time_zero - time_none:.6f} seconds")

/data/home/antoni/miniconda3/envs/svd/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


original sr 16000
torch.Size([2011819])
torch.Size([1, 16000])
torch.Size([1, 16000])
Outputs are equal
Max difference: 0.0
Mean difference: 0.0
Time with None mask: 0.300119 seconds
Time with zero mask: 0.497740 seconds
Time difference: 0.197621 seconds


In [7]:
output_zero.shape
output_none.shape


torch.Size([1, 48, 768])